# Integrasi dengan GEE

In [ ]:
# Import Library Earth Engine
import ee

In [ ]:
# Melakukan Autentikasi Pengguna.
ee.Authenticate()

# Mengaktifkan GEE pada Google Colab
ee.Initialize(project='ee-222112167')

# Load Data

In [ ]:

import gdown
import pandas as pd

# Google Drive file ID
# data 2020 = https://docs.google.com/spreadsheets/d/1PNIV1MBl5NvfuB-JRv2l4Gxy85xfchyUHzfqNjubNJY/edit?usp=sharing
# data 2021 = https://docs.google.com/spreadsheets/d/14seNh-9T7afCAw4g4bNuPRFqx42s6AmGg2PPa_6qoi4/edit?usp=sharing
# data 2022 = https://docs.google.com/spreadsheets/d/14j4DpolTMW_4seVU5coJVYWR4rgSL5ozRM3RzylL7qU/edit?usp=sharing
# data 2023 = https://docs.google.com/spreadsheets/d/1LVo8LsUJRCqJjMtrBAoqWoBTVMT-tltinZYVygPnwis/edit?usp=sharing
# data 2024 = https://docs.google.com/spreadsheets/d/1Plber-7ovoH2r123w4c1SLEezcVmAO40ng_GnEfFNPA/edit?usp=sharing

# data titik total
# https://docs.google.com/spreadsheets/d/1NgefeYgP351RDAWFDIxaEYcOTE48hmr_VO7bU7DvQW0/edit?usp=sharing

file_id = '1NgefeYgP351RDAWFDIxaEYcOTE48hmr_VO7bU7DvQW0'
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Specify the output file name
output = 'var_x.xlsx'

gdown.download(url, output, quiet=False)
var_x = pd.read_excel('var_x.xlsx')
var_x


Downloading...
From (original): https://drive.google.com/uc?id=1NgefeYgP351RDAWFDIxaEYcOTE48hmr_VO7bU7DvQW0
From (redirected): https://docs.google.com/spreadsheets/d/1NgefeYgP351RDAWFDIxaEYcOTE48hmr_VO7bU7DvQW0/export?format=xlsx
To: /content/var_x.xlsx
256kB [00:00, 77.8MB/s]


,Long,Lat,provinsi,Awal,Akhir
0,95.240478,5.455520,Aceh,2024-10-01,2024-12-31
1,95.236928,5.452742,Aceh,2024-10-01,2024-12-31
2,95.234944,5.448756,Aceh,2024-10-01,2024-12-31
3,95.251344,5.551639,Aceh,2024-10-01,2024-12-31
4,95.246778,5.553650,Aceh,2024-10-01,2024-12-31
...,...,...,...,...,...
821,99.476596,3.352033,Sumatera Utara,2024-10-01,2024-12-31
822,99.484115,3.351044,Sumatera Utara,2024-10-01,2024-12-31
823,98.211218,4.133046,Sumatera Utara,2024-10-01,2024-12-31
824,98.209161,4.155913,Sumatera Utara,2024-10-01,2024-12-31


In [ ]:
'''
import pandas as pd
var_x = pd.read_excel('Data Tanggal Scrap 2021.xlsx')
var_x
'''

"\nimport pandas as pd\nvar_x = pd.read_excel('Data Tanggal Scrap 2021.xlsx')\nvar_x\n"

# Scraping sesuai tanggal

In [ ]:
# Cloud masking function for Sentinel-2.
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

In [ ]:

# Daftar koordinat (longitude, latitude)
coordinates = [(row['Long'], row['Lat'], row['Awal'], row['Akhir']) for index, row in var_x.iterrows()]

# Membuat list untuk menyimpan nilai
band_values = []  # Menyimpan nilai untuk semua band
cloud_percentage_values = []  # Menambahkan list untuk menyimpan persentase awan
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']


# Mendefinisikan citra Sentinel-2 dan mengambil data pada 5 titik
for lon, lat, Awal, Akhir in coordinates:
    # Membuat titik dari koordinat
    point = ee.Geometry.Point(lon, lat)

    # Flag untuk mengecek apakah kita mendapatkan data valid
    found_valid_data = False

    # Definisikan list threshold awan yang ingin dicoba
    cloud_thresholds = [40, 50, 60, 70, 80]

    # Coba untuk setiap threshold yang telah ditentukan
    for cloud_threshold in cloud_thresholds:
        print(f"Trying with cloud threshold: {cloud_threshold}%")

        # Mengambil citra dengan threshold cloud yang lebih ketat
        image_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(point) \
            .filterDate(Awal, Akhir) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_threshold)) \
            .map(mask_s2_clouds)

        # Cek jika koleksi citra tidak kosong
        if image_collection.size().getInfo() > 0:
            image_collection = image_collection.mean()  # Mengambil rata-rata citra jika ada lebih dari 1 citra

            # Mengambil nilai dari beberapa band
            band_values_dict = {}

            for band in bands:
                value = image_collection.select(band).reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=point,
                    scale=20,
                    maxPixels=1e9
                )
                # Menyimpan nilai jika ada
                band_values_dict[band] = value.get(band).getInfo() if value.get(band) else None

            # Jika semua nilai band adalah NaN, anggap data tidak valid
            if all(val is None for val in band_values_dict.values()):
                print(f"Data at cloud threshold {cloud_threshold}% is not valid. Trying next threshold.")
                continue

            # Menambahkan nilai-nilai band ke list
            band_values.append(band_values_dict)

            # Mendapatkan nilai persentase awan dari gambar pertama
            cloud_percentage_values.append(cloud_threshold)  # Menyimpan persentase awan

            found_valid_data = True
            break  # Keluar dari loop jika data ditemukan dengan threshold tersebut

    # Jika semua threshold gagal, tambahkan None di akhir
    if not found_valid_data:
        print(f"No valid data found for coordinates: ({lon}, {lat}) after all cloud thresholds.")
        band_values.append({band: None for band in bands})  # Menambahkan None untuk semua band
        cloud_percentage_values.append(None)  # Tambahkan None untuk persentase awan

# Memeriksa panjang band_values dan cloud_percentage_values agar sesuai dengan panjang koordinat
print(f"Length of band_values: {len(band_values)}")
print(f"Length of cloud_percentage_values: {len(cloud_percentage_values)}")
print(f"Length of coordinates: {len(coordinates)}")

# Pastikan panjang band_values dan cloud_percentage_values sesuai dengan panjang coordinates
while len(band_values) < len(coordinates):
    band_values.append({band: None for band in bands})
    cloud_percentage_values.append(None)

# Menyimpan hasil ke dalam variabel
for band in bands:
    var_x[band] = [band_value.get(band) for band_value in band_values]

var_x['awan'] = cloud_percentage_values  # Menambahkan kolom 'awan' dengan persentase awan


Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Data at cloud threshold 40% is not valid. Trying next threshold.
Trying with cloud threshold: 50%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with cloud threshold: 40%
Trying with

In [ ]:
var_x

,Long,Lat,provinsi,Awal,Akhir,B2,B3,B4,B5,B6,B7,B8,B8A,B11,B12,awan
0,95.240478,5.455520,Aceh,2024-01-01,2024-03-31,0.093725,0.088525,0.071162,0.078325,0.075488,0.078025,0.066625,0.076663,0.077150,0.069937,40
1,95.236928,5.452742,Aceh,2024-01-01,2024-03-31,0.075125,0.067237,0.055625,0.058913,0.056900,0.058612,0.053638,0.057762,0.057800,0.053013,40
2,95.234944,5.448756,Aceh,2024-01-01,2024-03-31,0.095700,0.092767,0.083450,0.089917,0.085400,0.086917,0.082183,0.084017,0.093983,0.080900,40
3,95.251344,5.551639,Aceh,2024-01-01,2024-03-31,0.111883,0.115617,0.095400,0.098550,0.096067,0.098383,0.095500,0.096133,0.090033,0.080600,40
4,95.246778,5.553650,Aceh,2024-01-01,2024-03-31,0.100329,0.109043,0.090943,0.094586,0.087271,0.088243,0.075029,0.085786,0.080214,0.072243,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,99.476596,3.352033,Sumatera Utara,2024-01-01,2024-03-31,0.116000,0.127900,0.129300,0.144500,0.125800,0.124000,0.115500,0.125200,0.098400,0.068900,50
822,99.484115,3.351044,Sumatera Utara,2024-01-01,2024-03-31,0.117600,0.141900,0.145200,0.148100,0.087700,0.091700,0.070100,0.069400,0.032600,0.026500,50
823,98.211218,4.133046,Sumatera Utara,2024-01-01,2024-03-31,0.075700,0.095460,0.071780,0.070960,0.038520,0.034040,0.034480,0.023820,0.022860,0.010340,40
824,98.209161,4.155913,Sumatera Utara,2024-01-01,2024-03-31,0.103480,0.118060,0.087520,0.087240,0.062800,0.061680,0.063680,0.051960,0.038780,0.031300,40


# Menyimpan Hasil Scraping

In [ ]:
!pip install xlsxwriter
import xlsxwriter

Output_Data = var_x

from google.colab import drive
drive.mount('/content/drive')

Output_Data.to_excel('/content/drive/My Drive/SKRIPSI/Data Fix/Hasil triwulan 1 2024.xlsx', engine='xlsxwriter')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 5.7 MB/s eta 0:00:00
Mounted at /content/drive
